In [1]:
import pandas as pd
import numpy as np
import fastparquet
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
def data_preprocessing(file_name):    
    df = pd.read_parquet(file_name)
    df['trip_duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime 
    df['duration'] = df['trip_duration'].apply(lambda x:x.total_seconds()/60)
    df = df[(df['duration'] >=1) & (df['duration'] <= 60)]


    return df

In [3]:
df_train = data_preprocessing("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet")
df_valid = data_preprocessing("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-02.parquet")

In [4]:
len(df_train), len(df_valid)

(2898906, 2938060)

In [12]:

cat_var = ['PULocationID','DOLocationID']
num_var = ['trip_distance']
# df_train[cat_var] =df_train[cat_var].astype(str)

Dv = DictVectorizer()

train_dict = df_train[cat_var + num_var].to_dict(orient='records')
X_train = Dv.fit_transform(train_dict)

val_dict = df_valid[cat_var + num_var].to_dict(orient='records')
X_val = Dv.transform(val_dict)


Y_train = df_train['duration'].values
Y_valid = df_valid['duration'].values






In [13]:
lr = LinearRegression()
lr.fit(X_train,Y_train)
y_pred = lr.predict(X_val)
mean_squared_error(Y_valid,y_pred, squared=False)


/home/codespace/.local/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


13.04567441110816

In [ ]:
sns.histplot(y_pred,label='predicted')
sns.histplot(Y_train,label='actual')

plt.legend()